##### Title: Exercice 6.2
##### Author: Jerock Kalala
##### Date: January 29th 2023
##### Modified By: --
##### Using Natural Language Processing (NLP)


1. Using the Amazon Alexa reviews dataset, build a logistic regression model to predict positive or negative feedback based on review text. Be sure to run a test with something random you create (out of sample). Remember: 1 is positive, 0 is negative.

In [161]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14})
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import os

#print(os.listdir("E:\Bellevue\Winter_2022\DSC 360 Data Mining Text Analytics an\Week_6")) # will print everything in the directory

In [162]:
#Read data from a tab separeted file, using a delimeter '\t' for separating feature values.
amz_df = pd.read_csv("E:\\Bellevue\\Winter_2022\\DSC 360 Data Mining Text Analytics an\\Week_6\\amazon_alexa.tsv", delimiter='\t', quoting=3)

amz_df.head()

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"""Sometimes while playing a game, you can answe...",1
3,5,31-Jul-18,Charcoal Fabric,"""I have had a lot of fun with this thing. My 4...",1
4,5,31-Jul-18,Charcoal Fabric,Music,1


First, We'll need to know the dataset and the data characteristics:

In [163]:
print("The shape is: ", amz_df.shape, "\n")

#Checking the type of data
amz_df.info()

The shape is:  (3150, 5) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3150 entries, 0 to 3149
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   rating            3150 non-null   int64 
 1   date              3150 non-null   object
 2   variation         3150 non-null   object
 3   verified_reviews  3150 non-null   object
 4   feedback          3150 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 123.2+ KB


In [164]:
#Here is the observation:
print("1. In our dataset, we have {} of reviews" .format(amz_df.shape[0]))
print("2. Feedback = 1 means a positive review and Feedback=0 means a negative review")

1. In our dataset, we have 3150 of reviews
2. Feedback = 1 means a positive review and Feedback=0 means a negative review


In [165]:
#Checking out the type of feedback the dataset has
print(amz_df['feedback'].unique(), "\n")

#Checking the statistics
amz_df.describe()

[1 0] 



,rating,feedback
count,3150.000000,3150.000000
mean,4.463175,0.918413
std,1.068506,0.273778
min,1.000000,0.000000
25%,4.000000,1.000000
50%,5.000000,1.000000
75%,5.000000,1.000000
max,5.000000,1.000000


Now, let's Separate the reviews (positive and negative) and the rating  into different dataframes:

In [166]:
#Positive reviews
positive = amz_df[amz_df['feedback']==1]
pos = len(positive)
#negative reviews
negative = amz_df[amz_df['feedback']==0]
neg = len(negative)

#rating
rate_1 = amz_df[amz_df['rating']== 1]
r1 = len(rate_1)
rate_2 = amz_df[amz_df['rating']== 2]
r2 = len(rate_2)
rate_3 = amz_df[amz_df['rating']== 3]
r3 = len(rate_3)
rate_4 = amz_df[amz_df['rating']== 4]
r4 = len(rate_4)
rate_5 = amz_df[amz_df['rating']== 5]
r5 = len(rate_5)

print('- The number of positive reviews is {} and the number of negative reviews is {}' .format(pos, neg))
print("We have: ")
print('- {} ratings 1' .format(r1))
print('- {} ratings 2' .format(r2))
print('- {} ratings 3' .format(r3))
print('- {} ratings 4' .format(r4))
print('- {} ratings 5' .format(r5))

- The number of positive reviews is 2893 and the number of negative reviews is 257
We have: 
- 161 ratings 1
- 96 ratings 2
- 152 ratings 3
- 455 ratings 4
- 2286 ratings 5


Now let's clean our data:

In [167]:
amz_df.head()

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"""Sometimes while playing a game, you can answe...",1
3,5,31-Jul-18,Charcoal Fabric,"""I have had a lot of fun with this thing. My 4...",1
4,5,31-Jul-18,Charcoal Fabric,Music,1


In [168]:
# Drop unnecessary columns
amz_df_new = amz_df[['verified_reviews', 'feedback']].copy()
amz_df_new.head()

,verified_reviews,feedback
0,Love my Echo!,1
1,Loved it!,1
2,"""Sometimes while playing a game, you can answe...",1
3,"""I have had a lot of fun with this thing. My 4...",1
4,Music,1


Removing punctuation

In [169]:
print(string.punctuation)
#NLTK to remove stopwords
nltk.download('stopwords')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jeroc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Applying the functions described to clean the data

In [170]:
def cleaning(message):
  text = [char for char in message if char not in string.punctuation]
  text = ''.join(text)
  text = text.lower()
  text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
  return text

amz_clean = amz_df_new['verified_reviews'].apply(cleaning)

In [171]:
print(amz_clean[100])

['great', 'fun', 'getting', 'know', 'functions', 'product', 'wow', 'family', 'fun', 'homework', 'help', 'talking', 'grandchildrenwho', 'also', 'echo', 'huge', 'bonus', 'cant', 'wait', 'learn']


In [172]:
print(amz_df_new['verified_reviews'] [100])

"Great fun getting to know all the functions of this product.  WOW -- family fun and homework help.  Talking with other grandchildren,who also have an Echo, is a HUGE bonus.  Can't wait to learn more and more and more"


In [175]:
amz_clean.head(100)

0                                          [love, echo]
1                                               [loved]
2     [sometimes, playing, game, answer, question, c...
3     [lot, fun, thing, 4, yr, old, learns, dinosaur...
4                                               [music]
                            ...                        
95                           [like, far, lot, try, yet]
96                [love, using, strictly, music, great]
97    [love, echo, love, size, material, speaker, qu...
98                                               [love]
99    [entire, family, loves, alexa, echo, she’s, pa...
Name: verified_reviews, Length: 100, dtype: object

We can confirm that the stopwords and punctuation were removed.
Now let's transform the text information into a vector with count vectorizer:

*Train-test split:

In [176]:
X = amz_df_new['verified_reviews'].values
y = amz_df_new['feedback'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

*Vectorize the review text:

In [177]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

#Train the logistic regression model:

In [178]:
logistic = LogisticRegression()
logistic.fit(X_train, y_train)

LogisticRegression()

#Evaluate the model:

In [179]:
#accuracy score test
from sklearn.metrics import accuracy_score

y_pred = logistic.predict(X_test)
accuracy_score(y_test, y_pred)

0.9365079365079365

In [180]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = logistic.predict(X_test)

# Confusion matrix
conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)

# Classification report
cla_rep = classification_report(y_test, y_pred)
print(cla_rep)

[[ 26  32]
 [  8 564]]
              precision    recall  f1-score   support

           0       0.76      0.45      0.57        58
           1       0.95      0.99      0.97       572

    accuracy                           0.94       630
   macro avg       0.86      0.72      0.77       630
weighted avg       0.93      0.94      0.93       630



We are dealing with a very unbalanced dataset - most of the reviews are positive reviews.

#Test with something random I created (out of sample): I created 3 reviews.

In [181]:
new_review = ['This product is amazing and exceeded my expectations', 'This product is terrible', 'I am happy with the prod']
new_review = vectorizer.transform(new_review)

new_pred = logistic.predict(new_review)
print(new_pred) # Output: 1 (positive), and 0 (negative0

[1 0 1]


In [182]:
print("Clearly we can see that 2 out of our 3 reviews are positive, 1 is negative. the result is satisfactory.")

Clearly we can see that 2 out of our 3 reviews are positive, 1 is negative. the result is satisfactory.
